In [ ]:
from sklearn.preprocessing import Normalizer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#TF stuff
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers


#Scikit
from sklearn.metrics import log_loss
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import KFold
from sklearn.decomposition import PCA
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

In [ ]:
train_features = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
train_targets = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
train_features_enc = pd.get_dummies(train_features, columns=['cp_type', 'cp_dose'], drop_first=True)
X = train_features_enc.iloc[:,1:].to_numpy()
y = train_targets.iloc[:,1:].to_numpy() 
test_features = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
test_features_enc = pd.get_dummies(test_features, columns=['cp_type', 'cp_dose'], drop_first=True)
X_test = test_features_enc.iloc[:,1:].to_numpy()

In [ ]:
def l3_res_model(input_shape, no_classes, lr, no_neurons, no_dropout):
    inputs = tf.keras.Input(shape=input_shape)
    x = layers.Dense(no_neurons, activation='sigmoid')(inputs)
    x = layers.BatchNormalization()(x)
    b_1 = layers.Dropout(no_dropout)(x)
    x = layers.Dense(no_neurons, activation='sigmoid')(b_1)
    x = layers.BatchNormalization()(x)
    b_2 = layers.Dropout(no_dropout)(x)
    x = layers.Dense(no_neurons, activation='sigmoid')(b_2)
    x = layers.BatchNormalization()(x)
    b_3 = layers.Dropout(no_dropout)(x)
    tot_op = tf.keras.layers.add([b_1, b_2, b_3])
    outputs = layers.Dense(no_classes, activation='sigmoid')(tot_op)
    model = tf.keras.Model(inputs, outputs)
    model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate = lr), metrics=['binary_crossentropy'])
    return model

In [ ]:
train_features = pd.read_csv('/kaggle/input/lish-moa/train_features.csv')
train_targets = pd.read_csv('/kaggle/input/lish-moa/train_targets_scored.csv')
train_features_enc = pd.get_dummies(train_features, columns=['cp_type', 'cp_dose'], drop_first=True)
X = train_features_enc.iloc[:,1:].to_numpy()
y = train_targets.iloc[:,1:].to_numpy() 
test_features = pd.read_csv('/kaggle/input/lish-moa/test_features.csv')
test_features_enc = pd.get_dummies(test_features, columns=['cp_type', 'cp_dose'], drop_first=True)
X_test = test_features_enc.iloc[:,1:].to_numpy()

In [ ]:
from sklearn.model_selection import train_test_split
import torch.optim as optim

preds = 0
for i in range(50):
    np.random.seed(i)
    X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.3)

    control_vehicle_mask = X_train[:,-2] == 0
    X_train = X_train[~control_vehicle_mask,:]
    y_train = y_train[~control_vehicle_mask]

    transformer = Normalizer().fit(X_train)
    X_train = transformer.transform(X_train)
    X_test = transformer.transform(X_test)
    X_val = transformer.transform(X_val)

    nnclf = l3_res_model((875,),206, 0.0005, 256, 0.3)
    hist = nnclf.fit(X_train, y_train, batch_size=512, epochs=50, validation_data=(X_val, y_val), verbose=0)

    preds += nnclf.predict(X_test) # list of preds per class

preds = preds/50
control_mask = X_test[:,-2]==0
preds[control_mask] = 0
    
new_col = np.reshape(np.array(test_features.iloc[:,0]), (-1, 1))
preds = np.concatenate((new_col, preds), 1)
submission = pd.DataFrame(preds, columns= train_targets.columns)
submission.to_csv('submission.csv', index=False, header=True)